<a href="https://colab.research.google.com/github/qzzcl/iOSHealthKitToCSV/blob/main/iOSHealthRawDataToCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas matplotlib

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from google.colab import files  # Importing files module
plt.style.use("fivethirtyeight")

# Add a button for file upload
print("Please upload your XML file:")
uploaded = files.upload()

# Get the filename of the uploaded file
file_name = next(iter(uploaded))

# Create element tree object
tree = ET.ElementTree(ET.fromstring(uploaded[file_name]))
# For every health record, extract the attributes
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]


Please upload your XML file:


Saving export.xml to export.xml


In [4]:
record_data = pd.DataFrame(record_list)

# proper type to dates
for col in ['creationDate', 'startDate', 'endDate']:
    record_data[col] = pd.to_datetime(record_data[col])

# value is numeric, NaN if fails
record_data['value'] = pd.to_numeric(record_data['value'], errors='coerce')

# some records do not measure anything, just count occurences
# filling with 1.0 (= one time) makes it easier to aggregate
record_data['value'] = record_data['value'].fillna(1.0)

# shorter observation names
record_data['type'] = record_data['type'].str.replace('HKQuantityTypeIdentifier', '')
record_data['type'] = record_data['type'].str.replace('HKCategoryTypeIdentifier', '')
record_data.tail()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
518230,HeartRateVariabilitySDNN,Yordan’s Apple Watch,10.3.1,ms,2024-03-21 01:51:02+02:00,2024-03-21 01:50:02+02:00,2024-03-21 01:51:01+02:00,10.0599,"<<HKDevice: 0x28044be80>, name:Apple Watch, ma..."
518231,HeartRateVariabilitySDNN,Yordan’s Apple Watch,10.3.1,ms,2024-03-21 03:50:57+02:00,2024-03-21 03:49:55+02:00,2024-03-21 03:50:55+02:00,17.7633,"<<HKDevice: 0x28044be80>, name:Apple Watch, ma..."
518232,HeartRateVariabilitySDNN,Yordan’s Apple Watch,10.3.1,ms,2024-03-21 05:51:27+02:00,2024-03-21 05:50:26+02:00,2024-03-21 05:51:26+02:00,53.1558,"<<HKDevice: 0x28044be80>, name:Apple Watch, ma..."
518233,HeartRateVariabilitySDNN,Yordan’s Apple Watch,10.3.1,ms,2024-03-21 07:54:25+02:00,2024-03-21 07:53:24+02:00,2024-03-21 07:54:20+02:00,56.7411,"<<HKDevice: 0x28044be80>, name:Apple Watch, ma..."
518234,HeartRateVariabilitySDNN,Yordan’s Apple Watch,10.3.1,ms,2024-03-21 10:13:24+02:00,2024-03-21 10:12:23+02:00,2024-03-21 10:13:19+02:00,52.1438,"<<HKDevice: 0x28044be80>, name:Apple Watch, ma..."


In [ ]:
# Filter records for heart rate data
heart_rate_data = record_data[record_data['type'] == 'HeartRate']

# Plot heart rate data
plt.figure(figsize=(10, 6))
plt.plot(heart_rate_data['startDate'], heart_rate_data['value'], marker='o', linestyle='-')
plt.title('Heart Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Heart Rate')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()




In [13]:
from google.colab import files
files.download('heart_rate_data.csv')


# Select only the 'startDate' and 'value' columns
heart_rate_data_export = heart_rate_data[['startDate', 'value']]

# Rename the columns
heart_rate_data_export.columns = ['Date', 'HeartRate']

# Export to CSV
heart_rate_data_export.to_csv('heart_rate_data.csv', index=False)

print("Heart rate data exported to heart_rate_data.csv successfully.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Heart rate data exported to heart_rate_data.csv successfully.
